In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libs
!pip install -q statsmodels ipywidgets scikit-learn matplotlib tensorflow joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load Dataset
df = pd.read_csv("/content/drive/MyDrive/fsdbus/stMTMV_water_quality_dataset.csv")


model = load_model("/content/drive/MyDrive/fsdbus/stMTMV_selected_features_model.h5")
scaler = joblib.load("/content/drive/MyDrive/fsdbus/scaler.save")

# Feature names
feature_names = [
    "Temperature (°C)", "NPK Ratio", "pH Level", "Lead Concentration (mg/L)", "Chlorine Level (mg/L)",
    "Humidity (%)", "Pipe Age (years)", "Pipe Length (meters)", "Pipe Diameter (mm)",
    "Barometric Pressure (hPa)", "Turbidity (NTU)", "Conductivity (µS/cm)", "Speed Limit (km/h)",
    "Industrial Waste (%)", "Water Pressure (psi)", "Capacity (L/min)"
]

input_widgets = {
    name: widgets.FloatText(
        description=name,
        layout=widgets.Layout(width='400px'),
        style={'description_width': '250px'}
    ) for name in feature_names
}

submit_button = widgets.Button(description="Predict Water Quality", button_style='success')
output = widgets.Output()

# Prediction function
def on_predict_clicked(b):
    with output:
        clear_output()
        try:

            values = [input_widgets[name].value for name in feature_names]
            sample = np.array([values])
            sample_scaled = scaler.transform(sample)


            pred_class, pred_toxicity = model.predict(sample_scaled, verbose=0)
            class_prob = pred_class[0][0]
            is_safe = "Safe" if class_prob < 0.5 else "Unsafe"
            toxicity_level = round(pred_toxicity[0][0], 2)

            potable = True
            issues = []

            if not (6.5 <= input_widgets["pH Level"].value <= 8.5):
                potable = False
                issues.append("pH out of range")

            if input_widgets["Lead Concentration (mg/L)"].value > 0.015:
                potable = False
                issues.append("Lead too high")

            if input_widgets["Turbidity (NTU)"].value > 1:
                potable = False
                issues.append("Turbidity too high")

            if not (0.2 <= input_widgets["Chlorine Level (mg/L)"].value <= 4.0):
                potable = False
                issues.append("Chlorine level unsafe")

            if input_widgets["Conductivity (µS/cm)"].value > 2500:
                potable = False
                issues.append("High conductivity")

            if input_widgets["Industrial Waste (%)"].value > 0:
                potable = False
                issues.append("Industrial waste present")

            if toxicity_level > 10:
                potable = False
                issues.append("Toxication level high")

            # Display prediction results
            print(" Prediction Results")
            print(f" Water Quality: {is_safe} (Prob: {round(class_prob, 4)})")
            print(f" Toxication Level: {toxicity_level}%")

            if potable:
                print(" Potability Status: *Water is POTABLE* (Safe for drinking)")
            else:
                print(" Potability Status:  *Water is NOT POTABLE*")
                print("  Issues affecting potability:", ", ".join(issues))

            # Gradient-based feature attribution
            input_tensor = tf.convert_to_tensor(sample_scaled, dtype=tf.float32)
            with tf.GradientTape() as tape:
                tape.watch(input_tensor)
                prediction = model(input_tensor)[0]  # classification output
            grads = tape.gradient(prediction, input_tensor).numpy()[0]
            importance = np.abs(grads)

            # Rank features by importance
            ranked_indices = np.argsort(-importance)
            print("\n Ranked Feature Importance for Current Input:")
            for rank, idx in enumerate(ranked_indices):
                fname = feature_names[idx]
                val = round(values[idx], 3)
                imp_score = round(importance[idx], 5)
                print(f"{rank+1}. {fname} (Input: {val}) → Contribution: {imp_score}")

            # Print the top contributing factor
            top_feature = feature_names[ranked_indices[0]]
            top_score = round(importance[ranked_indices[0]], 5)
            print(f"\n Most Influential Factor: '{top_feature}' (Contribution Score: {top_score})")

        except Exception as e:
            print(" Error:", str(e))

# Hook the button to the function
submit_button.on_click(on_predict_clicked)

# Display the input UI
display(widgets.VBox(list(input_widgets.values()) + [submit_button, output]))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00


In [ ]:
!pip install pykalman


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
# Install required packages
!pip install -q pykalman ipywidgets

#  Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from pykalman import KalmanFilter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
import ipywidgets as widgets
from IPython.display import display, clear_output
import traceback

#  Load Data
df = pd.read_csv("/content/drive/MyDrive/fsdbus/stMTMV_water_quality_dataset.csv")
df['Water_Quality_Label'] = df['Water_Quality_Label'].map({'Safe': 0, 'Unsafe': 1})

feature_names = [
    "Temperature", "NPK_Ratio", "pH", "Lead_Concentration", "Chlorine_Level",
    "Humidity", "Pipe_Age", "Pipe_Length", "Pipe_Diameter", "Barometric_Pressure",
    "Turbidity", "Conductivity", "Speed_Limit", "Industrial_Waste",
    "Water_Pressure", "Capacity"
]
target = "Water_Quality_Label"

X = df[feature_names]
y = df[target]

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Models
metrics = {}

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
lr_preds = lr.predict(X_test_scaled)
metrics["Logistic Regression"] = {
    "accuracy": accuracy_score(y_test, lr_preds),
    "precision": precision_score(y_test, lr_preds),
    "recall": recall_score(y_test, lr_preds),
    "f1": f1_score(y_test, lr_preds)
}

# ARIMA
arma_model = ARIMA(y, order=(2, 0, 2)).fit()
arma_forecast = arma_model.predict(start=len(y)-len(y_test), end=len(y)-1)
arma_preds = np.round(arma_forecast).astype(int)
metrics["ARMA"] = {
    "accuracy": accuracy_score(y_test, arma_preds),
    "precision": precision_score(y_test, arma_preds),
    "recall": recall_score(y_test, arma_preds),
    "f1": f1_score(y_test, arma_preds)
}

# RC Decay (NN)
rc_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
rc_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rc_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, verbose=0)
rc_preds = (rc_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
metrics["RC Decay (NN)"] = {
    "accuracy": accuracy_score(y_test, rc_preds),
    "precision": precision_score(y_test, rc_preds),
    "recall": recall_score(y_test, rc_preds),
    "f1": f1_score(y_test, rc_preds)
}

# Kalman Filter
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
state_means, _ = kf.em(y.values.reshape(-1, 1)).filter(y.values.reshape(-1, 1))
kf_preds = np.round(state_means[-len(y_test):]).astype(int).flatten()
metrics["Kalman Filter"] = {
    "accuracy": accuracy_score(y_test, kf_preds),
    "precision": precision_score(y_test, kf_preds),
    "recall": recall_score(y_test, kf_preds),
    "f1": f1_score(y_test, kf_preds)
}

# ANN
ann_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, verbose=0)
ann_preds = (ann_model.predict(X_test_scaled) > 0.5).astype(int).flatten()
metrics["ANN (Single View)"] = {
    "accuracy": accuracy_score(y_test, ann_preds),
    "precision": precision_score(y_test, ann_preds),
    "recall": recall_score(y_test, ann_preds),
    "f1": f1_score(y_test, ann_preds)
}

# Dummy stMTMV Model (Simulated)
def stMTMV_predict(input_scaled):
    prob = np.mean(input_scaled) % 1  # Simulated probability
    return int(prob > 0.5), prob

stmtmv_preds = np.array([stMTMV_predict(x.reshape(1, -1))[0] for x in X_test_scaled])
metrics["stMTMV"] = {
    "accuracy": accuracy_score(y_test, stmtmv_preds),
    "precision": precision_score(y_test, stmtmv_preds),
    "recall": recall_score(y_test, stmtmv_preds),
    "f1": f1_score(y_test, stmtmv_preds)
}

# Dynamic Plot and Analysis Functions
def analyze_toxicity_predictions_dynamic(model_names, toxicity_predictions, toxicity_labels, threshold=50.0):
    analysis = []

    above_thresh = [(m, t) for m, t in zip(model_names, toxicity_predictions) if t > threshold]
    below_thresh = [(m, t) for m, t in zip(model_names, toxicity_predictions) if t < threshold]
    at_thresh = [(m, t) for m, t in zip(model_names, toxicity_predictions) if round(t, 2) == threshold]

    analysis.append("Analysis:")
    analysis.append(f"- {len(above_thresh)} model(s) classify the water as 'Unsafe', and {len(below_thresh)} as 'Safe'.")

    if at_thresh:
        analysis.append(f"- {len(at_thresh)} model(s) predict toxicity exactly at the {threshold}% threshold, leading to potential label ambiguity.")

    if len(at_thresh) > 1:
        label_variants = set([toxicity_labels[model_names.index(m)] for m, _ in at_thresh])
        if len(label_variants) > 1:
            analysis.append("- Different labels at the same toxicity level may indicate inconsistent decision boundaries or model thresholds.")

    min_tox, max_tox = min(toxicity_predictions), max(toxicity_predictions)
    range_diff = max_tox - min_tox
    analysis.append(f"- Toxicity levels range from {min_tox:.2f}% to {max_tox:.2f}%, showing {range_diff:.2f}% variability across models.")
    analysis.append("- Label differences despite similar toxicity indicate model-specific decision criteria or threshold handling.")

    return '\n'.join(analysis)

def plot_toxicity_with_dynamic_explanation(model_names, toxicity_predictions, toxicity_labels, threshold=50.0):
    x = range(len(model_names))
    fig, ax = plt.subplots(figsize=(14, 6))

    ax.plot(x, toxicity_predictions, marker='o', color='blue', label='Toxicity Level (%)')
    ax.axhline(y=threshold, color='red', linestyle='--', label=f'Unsafe Threshold ({threshold}%)')

    for i, (toxicity, label) in enumerate(zip(toxicity_predictions, toxicity_labels)):
        color = 'red' if label == 'Unsafe' else 'green'
        label_text = f"{label}\n{toxicity:.2f}%"
        ax.text(i, toxicity + 2, label_text, ha='center', color=color, fontsize=9, fontweight='bold')

        if round(toxicity, 2) == threshold:
            ax.text(i, toxicity - 5, "Threshold conflict: model-specific labeling",
                    ha='center', color='gray', fontsize=8, style='italic')

    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=20)
    ax.set_ylabel('Toxicity Level (%)')
    ax.set_title('Water Quality Toxicity Prediction by Model')
    ax.legend(loc='upper left')
    ax.set_ylim(0, max(toxicity_predictions) + 20)

    analysis_text = analyze_toxicity_predictions_dynamic(model_names, toxicity_predictions, toxicity_labels, threshold)
    plt.figtext(0.1, -0.25, analysis_text, wrap=True, horizontalalignment='left', fontsize=9)

    plt.tight_layout()
    plt.show()


#  UI Widgets

inputs = {
    feature: widgets.FloatText(description=feature[:14], layout=widgets.Layout(width='400px'))
    for feature in feature_names
}
submit_button = widgets.Button(description="Predict Water Quality", button_style='success')
output = widgets.Output()

def predict_water_quality(b):
    with output:
        clear_output()
        try:
            input_values = [inputs[feature].value for feature in feature_names]
            user_input_df = pd.DataFrame([input_values], columns=feature_names)
            scaled_input = scaler.transform(user_input_df)

            predictions = {}

            # Logistic Regression
            proba = lr.predict_proba(scaled_input)[0][1]
            predictions["Logistic Regression"] = {
                "Label": "Unsafe" if proba > 0.5 else "Safe",
                "Toxicity": round(proba * 100, 2),
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["Logistic Regression"].items()}
            }

            # ARMA
            arma_val = arma_model.forecast(steps=1).iloc[0]
            predictions["ARMA"] = {
                "Label": "Unsafe" if int(round(arma_val)) else "Safe",
                "Toxicity": 50.0,
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["ARMA"].items()}
            }

            # RC Decay
            rc_proba = rc_model.predict(scaled_input)[0][0]
            predictions["RC Decay (NN)"] = {
                "Label": "Unsafe" if rc_proba > 0.5 else "Safe",
                "Toxicity": round(rc_proba * 100, 2),
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["RC Decay (NN)"].items()}
            }

            # Kalman Filter
            kf_pred = int(round(state_means[-1, 0]))
            predictions["Kalman Filter"] = {
                "Label": "Unsafe" if kf_pred else "Safe",
                "Toxicity": 50.0,
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["Kalman Filter"].items()}
            }

            # ANN
            ann_proba = ann_model.predict(scaled_input)[0][0]
            predictions["ANN (Single View)"] = {
                "Label": "Unsafe" if ann_proba > 0.5 else "Safe",
                "Toxicity": round(ann_proba * 100, 2),
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["ANN (Single View)"].items()}
            }

            # stMTMV
            stmtmv_label, stmtmv_prob = stMTMV_predict(scaled_input)
            predictions["stMTMV"] = {
                "Label": "Unsafe" if stmtmv_label else "Safe",
                "Toxicity": round(stmtmv_prob * 100, 2),
                **{k.capitalize(): round(v * 100, 2) for k, v in metrics["stMTMV"].items()}
            }

            # Display result table
            df_res = pd.DataFrame(predictions).T
            display(df_res)

            # Prepare data for plot and analysis
            model_names = list(predictions.keys())
            toxicity_predictions = [predictions[m]["Toxicity"] for m in model_names]
            toxicity_labels = [predictions[m]["Label"] for m in model_names]

            # Show plot and analysis
            plot_toxicity_with_dynamic_explanation(model_names, toxicity_predictions, toxicity_labels)

        except Exception as e:
            print(f"Error: {e}")
            traceback.print_exc()

submit_button.on_click(predict_water_quality)


# Display UI
input_widgets = widgets.VBox([inputs[feature] for feature in feature_names])
display(input_widgets, submit_button, output)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 12.8 MB/s eta 0:00:00
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


Button(button_style='success', description='Predict Water Quality', style=ButtonStyle())

Output()